In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_47/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
nltk=3.5 already installed
unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


<img src='../../../common/logo_DH.png' align='left' width=35%/>

### Checkpoint: Clasificacion de texto

Trabajaremos con un dataset de noticias (en ingles) sobre diferentes temas: world, sports, business, sci/tech.

La idea es vectorizar el corpus de noticias e implementar un clasificador para identificar a qué tema pertenecen las noticias.

Trabajaremos con una versión reducida de un corpus que pueden descargar completo (~30Mb) del siguiente sitio:

https://github.com/mhjabreel/CharCnn_Keras

#### Ejercicio 1

Generamos el corpus:

Importen los datos con pandas. Los datos se encuentran en '../Data/ag_news_reduced.csv'. 

El dataset contiene cuatro columnas: la primera tiene un entero que indica a qué clase pertenece la noticia. La segunda es el nombre de la clase, la tercera es el título y la cuarta es una descripción de la noticia.

Como no tenemos las noticias enteras sino solo un resumen, conviene que generemos el corpus concatenando el título y la descripción en un sólo texto. 

Generen una columna en el dataframe que sea la concatenación del título y la descripción. No olviden agregar un espacio en blanco para no pegar la última palabra del título con la primera de la descrición.

¿Cuántos artículos hay de cada clase?

In [2]:
import pandas as pd

data=pd.read_csv('../Data/ag_news_reduced.csv');

data.head()

,class,class_name,title,description
0,3,Business,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Business,Google IPO faces Playboy slip-up,The bidding gets underway for Google's public ...
2,3,Business,Letters,Target the abusers of legal weapons We can all...
3,3,Business,Oldsmobile: The final parking lot,Why General Motors dropped the Oldsmobile. The...
4,4,Sci/Tech,AOL to Sell Cheap PCs to Minorities and Senior...,Reuters - America Online on Thursday said it\p...


In [3]:
data['news']=data['title']+ ' '+data['description'];

In [4]:
data['news'][0]

"Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."

In [5]:
data['class_name'].value_counts()

Sports      1539
Sci/Tech    1520
World       1486
Business    1455
Name: class_name, dtype: int64

In [6]:
len(data)

6000

#### Ejercicio 2

¿Cuáles son las palabras más frecuentes dentro de cada clase?

Ayuda: Pueden vectorizar el corpus (dividido por temas) usando CountVectorizer() y luego sumar las filas de la matriz para obtener el número total de veces que aparece cada término.

Ayuda2: No olviden remover stopwords al vectorizar!

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np
stop_words=stopwords.words('english');

vectorizer=CountVectorizer(stop_words=stop_words);

clases=['World','Sports','Business','Sci/Tech'];

for clase in range(1,5):
    X=vectorizer.fit_transform(data[data['class']==clase]['news']);
    counts=X.sum(axis=0);
    counts=np.array(counts);
    
    indices=np.argsort(counts);
    valores=np.sort(counts);
    indices=indices[0][::-1];
    valores=valores[0][::-1];
    terms=np.array(vectorizer.get_feature_names());

    print('\n Clase ',clases[clase-1])
    print(terms[indices[:20]])


 Clase  World
['39' 'said' 'iraq' 'ap' 'reuters' 'president' 'us' 'two' 'afp' 'minister'
 'people' 'new' 'quot' 'killed' 'government' 'baghdad' 'bush' 'iraqi'
 'palestinian' 'prime']

 Clase  Sports
['39' 'ap' 'new' 'first' 'game' 'team' 'two' 'season' 'win' 'one' 'night'
 'cup' 'world' 'year' 'league' 'sunday' 'last' 'victory' 'quot' 'time']

 Clase  Business
['39' 'reuters' 'said' 'oil' 'new' 'us' 'gt' 'lt' 'stocks' 'inc' 'company'
 'prices' 'fullquote' 'york' 'corp' 'monday' 'percent' 'sales' 'year'
 'tuesday']

 Clase  Sci/Tech
['39' 'new' 'microsoft' 'software' 'said' 'ap' 'lt' 'gt' 'reuters'
 'internet' 'company' 'space' 'music' 'search' 'quot' 'inc' 'security'
 'world' 'technology' 'com']


#### Ejercicio 3

Vectorizar el corpus usando TfidfVectorizer y entrenar un clasificador de tipo Multinomial Naive Bayes (MultinomialNB). Para empezar, hagan un simple train_test_split de los datos y vean la performance (accuracy) en el set de validacion.

Luego vean si pueden mejorar esa performance optimizando el parámetro alpha del modelo haciendo una Gridsearch cross validation.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split
from sklearn.metrics import accuracy_score

train,test=train_test_split(data,stratify=data['class'],random_state=3);

vectorizer=TfidfVectorizer();
X_train=vectorizer.fit_transform(train['news']);
y_train=train['class'];

X_test=vectorizer.transform(test['news']);
y_test=test['class'];

nbc=MultinomialNB();

nbc.fit(X_train,y_train);
y_pred=nbc.predict(X_test);

print('Accuracy:',accuracy_score(y_test,y_pred))



Accuracy: 0.8686666666666667


In [9]:
# Optimizamos en alpha

vectorizer=TfidfVectorizer();

X=vectorizer.fit_transform(data['news']);
y=data['class'];

skf=StratifiedKFold(n_splits=3,random_state=3,shuffle=True);

params={'alpha':np.arange(0.01,1,0.05)};
GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=3,iid=False);
GS_CV.fit(X,y);
print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.8835000000000001
best params: {'alpha': 0.16000000000000003}


[Parallel(n_jobs=3)]: Done  60 out of  60 | elapsed:    2.2s finished
/Users/csuarezgurruchaga/opt/anaconda3/envs/dhdsblend2021/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


In [10]:
X.shape

(6000, 19590)

#### Ejercicio 4

Repetir lo anterior removiendo stopwords al vectorizar.

¿Cambia la performance?  ¿Cuánto se redujo la dimensionalidad?

In [11]:
# Include Stopwords
from nltk.corpus import stopwords
stop_words=stopwords.words('english');
stop_words.append('39');
stop_words.append('reuters');

vectorizer=TfidfVectorizer(stop_words=stop_words);

X_train=vectorizer.fit_transform(data['news']);
y_train=data['class'];

skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True);
params={'alpha':np.arange(0.01,1,0.05)};
GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=3,iid=False);
GS_CV.fit(X_train,y_train);
print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.8856666666666667
best params: {'alpha': 0.41000000000000003}


[Parallel(n_jobs=3)]: Done  60 out of  60 | elapsed:    0.2s finished
/Users/csuarezgurruchaga/opt/anaconda3/envs/dhdsblend2021/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


In [12]:
print(X_train.shape)

(6000, 19453)


#### Ejercicio 5

Utilizar los parámetros min_df y max_df para remover términos que aparecen en muy pocos documentos o que aparecen en demasiados (stopwords específicas del corpus).

Reentrenar el modelo y ver si cambia la performance.



In [13]:
# min_df

vectorizer=TfidfVectorizer(stop_words=stop_words,min_df=2,max_df=0.8);
X_train=vectorizer.fit_transform(data['news']);
y_train=data['class'];

print(X_train.shape)

skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True);
params={'alpha':np.arange(0.1,1,0.05)};
GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=3,iid=False);
GS_CV.fit(X_train,y_train);
print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)


(6000, 10430)
Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


best score: 0.8858333333333333
best params: {'alpha': 0.3500000000000001}


[Parallel(n_jobs=3)]: Done  54 out of  54 | elapsed:    0.2s finished
/Users/csuarezgurruchaga/opt/anaconda3/envs/dhdsblend2021/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


#### Ejercicio 6

Repetir el análisis incluyendo bigramas en la vectorizacion.

In [14]:
# n-grams

vectorizer=TfidfVectorizer(stop_words=stop_words,min_df=2,ngram_range=(1,2));

X_train=vectorizer.fit_transform(data['news']);
y_train=data['class'];

print(X_train.shape)

skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True);
params={'alpha':np.arange(0.05,0.15,0.01)};
GS_CV=GridSearchCV(MultinomialNB(),params,cv=skf,verbose=1,n_jobs=3,iid=False);
GS_CV.fit(X_train,y_train);
print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)

(6000, 21541)
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed:    0.1s finished
/Users/csuarezgurruchaga/opt/anaconda3/envs/dhdsblend2021/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


best score: 0.8886666666666666
best params: {'alpha': 0.09000000000000001}
